Here I tried training the dataset given to me on YOLO v8 instance segmentation model but since there is a certain manner of dataset storing , i had to convert my masked labels to bounding box coordinates and save them in .txt extension for training.


In [ ]:
import cv2
import numpy as np
import os

def mask_to_bbox(mask_path):
    # Load the mask image in grayscale
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise ValueError("Mask image could not be loaded.")

    # Find contours from the binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes = []

    for contour in contours:
        # Calculate the bounding rectangle for each contour
        x, y, w, h = cv2.boundingRect(contour)
        boxes.append((x, y, w, h))

    return boxes

def convert_to_yolov8_format(bbox, image_shape):
    img_height, img_width = image_shape
    yolov8_data = []

    for (x, y, w, h) in bbox:
        x_center = ((x + w / 2) / img_width)
        y_center = ((y + h / 2) / img_height)
        width = (w / img_width)
        height = (h / img_height)
        # Assuming class_id is 0 for all, change it as needed
        yolov8_data.append(f"0 {x_center} {y_center} {width} {height}")

    return yolov8_data

def process_masks(input_dir, output_dir, image_shape):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):  # Assuming mask images are PNG
            mask_path = os.path.join(input_dir, filename)
            bbox = mask_to_bbox(mask_path)
            yolov8_format = convert_to_yolov8_format(bbox, image_shape)

            # Create a .txt file for each mask
            base_filename = os.path.splitext(filename)[0]
            output_filepath = os.path.join(output_dir, f"{base_filename}.txt")
            with open(output_filepath, 'w') as file:
                for line in yolov8_format:
                    file.write(line + "\n")

# Example usage
input_directory = '/kaggle/input/vineyard32/masks/masks'  # Specify the path to your mask images
output_directory = '/kaggle/working/box_mask'  # Specify where to save the output .txt files
image_width, image_height = 1920, 960  # Set the size of your images

process_masks(input_directory, output_directory, (image_height, image_width))


In [ ]:
import zipfile
import os


In [ ]:
def zip_folder(folder_path, output_path):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a complete filepath of file in directory
                file_path = os.path.join(root, file)
                # Add file to zip
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Example usage
folder_to_zip = '/kaggle/working/box_mask'  # Folder containing the .txt files
zip_output_path = '/kaggle/working/mask.zip'  # Path where the zip file will be saved

zip_folder(folder_to_zip, zip_output_path)


In [ ]:
!pip install ultralytics==8.0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
!yolo task=segment mode=train model=yolov8s-seg.pt  data=/content/drive/MyDrive/data.yaml epochs=10 imgsz=640

Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=segment, mode=train, model=yolov8s-seg.pt, data=/content/drive/MyDrive/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=N

In [ ]:
import os

# Paths to your directories
image_dir = '/content/drive/MyDrive/Train/images'
label_dir = '/content/drive/MyDrive/Train/mask'

# Get list of files
images = os.listdir(image_dir)
labels = os.listdir(label_dir)

# Sort lists to maintain order
images.sort()
labels.sort()

print("Images:", images)
print("Labels:", labels)


Images: ['Block_1C1_Row_1_Middle_1003.png', 'Block_1C1_Row_1_Middle_1013.png', 'Block_1C1_Row_1_Middle_1018.png', 'Block_1C1_Row_1_Middle_1023.png', 'Block_1C1_Row_1_Middle_1048.png', 'Block_1C1_Row_1_Middle_1058.png', 'Block_1C1_Row_1_Middle_1063.png', 'Block_1C1_Row_1_Middle_1068.png', 'Block_1C1_Row_1_Middle_1073.png', 'Block_1C1_Row_1_Middle_1078.png', 'Block_1C1_Row_1_Middle_1083.png', 'Block_1C1_Row_1_Middle_1103 (1).png', 'Block_1C1_Row_1_Middle_1103.png', 'Block_1C1_Row_1_Middle_1108.png', 'Block_1C1_Row_1_Middle_1113.png', 'Block_1C1_Row_1_Middle_1133.png', 'Block_1C1_Row_1_Middle_1138.png', 'Block_1C1_Row_1_Middle_1143.png', 'Block_1C1_Row_1_Middle_1148.png', 'Block_1C1_Row_1_Middle_1153.png', 'Block_1C1_Row_1_Middle_1158.png', 'Block_1C1_Row_1_Middle_1163.png', 'Block_1C1_Row_1_Middle_1173.png', 'Block_1C1_Row_1_Middle_1178.png', 'Block_1C1_Row_1_Middle_1183.png', 'Block_1C1_Row_1_Middle_1213.png', 'Block_1C1_Row_1_Middle_1218.png', 'Block_1C1_Row_1_Middle_1238.png', 'Block_

In [ ]:
def replace_extension(filename, new_extension):
    """Replace the file extension with a new one."""
    base = os.path.splitext(filename)[0]  # Get the file name without extension
    return f"{base}{new_extension}"

# Assuming image files are .jpg and label files are .txt
new_labels = [replace_extension(img, '.txt') for img in images]

# Dictionary to map old label names to new ones
label_map = dict(zip(labels, new_labels))

# Now, rename the label files
for old_label, new_label in label_map.items():
    old_label_path = os.path.join(label_dir, old_label)
    new_label_path = os.path.join(label_dir, new_label)
    os.rename(old_label_path, new_label_path)

print("Renaming complete.")


Renaming complete.


In [ ]:
updated_labels = os.listdir(label_dir)
updated_labels.sort()
print("Updated Labels:", updated_labels)


Updated Labels: ['Block_1C1_Row_1_Middle_1003.txt', 'Block_1C1_Row_1_Middle_1013.txt', 'Block_1C1_Row_1_Middle_1018.txt', 'Block_1C1_Row_1_Middle_1023.txt', 'Block_1C1_Row_1_Middle_1048.txt', 'Block_1C1_Row_1_Middle_1058.txt', 'Block_1C1_Row_1_Middle_1063.txt', 'Block_1C1_Row_1_Middle_1068.txt', 'Block_1C1_Row_1_Middle_1073.txt', 'Block_1C1_Row_1_Middle_1078.txt', 'Block_1C1_Row_1_Middle_1083.txt', 'Block_1C1_Row_1_Middle_1103 (1).txt', 'Block_1C1_Row_1_Middle_1103.txt', 'Block_1C1_Row_1_Middle_1108.txt', 'Block_1C1_Row_1_Middle_1113.txt', 'Block_1C1_Row_1_Middle_1133.txt', 'Block_1C1_Row_1_Middle_1138.txt', 'Block_1C1_Row_1_Middle_1143.txt', 'Block_1C1_Row_1_Middle_1148.txt', 'Block_1C1_Row_1_Middle_1153.txt', 'Block_1C1_Row_1_Middle_1158.txt', 'Block_1C1_Row_1_Middle_1163.txt', 'Block_1C1_Row_1_Middle_1173.txt', 'Block_1C1_Row_1_Middle_1178.txt', 'Block_1C1_Row_1_Middle_1183.txt', 'Block_1C1_Row_1_Middle_1213.txt', 'Block_1C1_Row_1_Middle_1218.txt', 'Block_1C1_Row_1_Middle_1238.txt',

In [ ]:
import os

# Paths to your directories
image_dir = '/content/drive/MyDrive/Val/val_images'
label_dir = '/content/drive/MyDrive/Val/val_masks'

# Get list of files
images = os.listdir(image_dir)
labels = os.listdir(label_dir)

# Sort lists to maintain order
images.sort()
labels.sort()

print("Images:", images)
print("Labels:", labels)


Images: ['Block_5E1_Row_1_Middle_1871.png', 'Block_5E1_Row_1_Middle_1911.png', 'Block_5E1_Row_1_Middle_1919.png', 'Block_5E1_Row_1_Middle_1943.png', 'Block_5E1_Row_1_Middle_1959.png', 'Block_5E1_Row_1_Middle_1975.png', 'Block_5E1_Row_1_Middle_1983.png', 'Block_5E1_Row_1_Middle_1991.png', 'Block_5E1_Row_1_Middle_1999.png', 'Block_5E1_Row_1_Middle_2023.png', 'Block_5E1_Row_1_Middle_2039.png', 'Block_5E1_Row_1_Middle_2055.png', 'Block_5E1_Row_1_Middle_2087.png', 'Block_5E1_Row_1_Middle_2095.png', 'Block_5E1_Row_1_Middle_2103.png', 'Block_5E1_Row_1_Middle_2127.png', 'Block_5E1_Row_1_Middle_2135.png', 'Block_5E1_Row_1_Middle_2151.png', 'Block_5E1_Row_1_Middle_2183.png', 'Block_5E1_Row_1_Middle_2191.png', 'Block_5E1_Row_1_Middle_2199.png', 'Block_5E1_Row_1_Middle_2223.png', 'Block_5E1_Row_1_Middle_2231.png', 'Block_5E1_Row_1_Middle_2247.png', 'Block_5E1_Row_1_Middle_2271.png', 'Block_5E1_Row_1_Middle_2287.png', 'Block_5E1_Row_1_Middle_2295.png', 'Block_5E1_Row_1_Middle_2319.png', 'Block_5E1_

In [ ]:
def replace_extension(filename, new_extension):
    """Replace the file extension with a new one."""
    base = os.path.splitext(filename)[0]  # Get the file name without extension
    return f"{base}{new_extension}"

# Assuming image files are .jpg and label files are .txt
new_labels = [replace_extension(img, '.txt') for img in images]

# Dictionary to map old label names to new ones
label_map = dict(zip(labels, new_labels))

# Now, rename the label files
for old_label, new_label in label_map.items():
    old_label_path = os.path.join(label_dir, old_label)
    new_label_path = os.path.join(label_dir, new_label)
    os.rename(old_label_path, new_label_path)

print("Renaming complete.")


Renaming complete.


In [ ]:
updated_labels = os.listdir(label_dir)
updated_labels.sort()
print("Updated Labels:", updated_labels)


Updated Labels: ['Block_5E1_Row_1_Middle_1871.txt', 'Block_5E1_Row_1_Middle_1911.txt', 'Block_5E1_Row_1_Middle_1919.txt', 'Block_5E1_Row_1_Middle_1943.txt', 'Block_5E1_Row_1_Middle_1959.txt', 'Block_5E1_Row_1_Middle_1975.txt', 'Block_5E1_Row_1_Middle_1983.txt', 'Block_5E1_Row_1_Middle_1991.txt', 'Block_5E1_Row_1_Middle_1999.txt', 'Block_5E1_Row_1_Middle_2023.txt', 'Block_5E1_Row_1_Middle_2039.txt', 'Block_5E1_Row_1_Middle_2055.txt', 'Block_5E1_Row_1_Middle_2087.txt', 'Block_5E1_Row_1_Middle_2095.txt', 'Block_5E1_Row_1_Middle_2103.txt', 'Block_5E1_Row_1_Middle_2127.txt', 'Block_5E1_Row_1_Middle_2135.txt', 'Block_5E1_Row_1_Middle_2151.txt', 'Block_5E1_Row_1_Middle_2183.txt', 'Block_5E1_Row_1_Middle_2191.txt', 'Block_5E1_Row_1_Middle_2199.txt', 'Block_5E1_Row_1_Middle_2223.txt', 'Block_5E1_Row_1_Middle_2231.txt', 'Block_5E1_Row_1_Middle_2247.txt', 'Block_5E1_Row_1_Middle_2271.txt', 'Block_5E1_Row_1_Middle_2287.txt', 'Block_5E1_Row_1_Middle_2295.txt', 'Block_5E1_Row_1_Middle_2319.txt', 'Bl